<a href="https://colab.research.google.com/github/DmitryKutsev/eng_to_jap_translator/blob/main/my_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tinysegmenter

In [3]:
import sys
import os
import math
from tqdm import tqdm

import torch
import torch.optim as optim
import torch.nn as nn
import pandas as pd
import numpy as np

import torchtext
from torchtext.data import Field, BucketIterator, TabularDataset
import random
import spacy
import tinysegmenter

import torch
import torch.nn as nn
import random


In [4]:
# SEED = 1234

# random.seed(SEED)
# np.random.seed(SEED)
# torch.manual_seed(SEED)
# torch.cuda.manual_seed(SEED)
# torch.backends.cudnn.deterministic = True

In [5]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
spacy_en = spacy.load('en')

In [7]:
segmenter = tinysegmenter.TinySegmenter()

In [8]:
! wget https://nlp.stanford.edu/projects/jesc/data/raw.tar.gz

--2020-12-19 11:40:19--  https://nlp.stanford.edu/projects/jesc/data/raw.tar.gz
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102198198 (97M) [application/x-gzip]
Saving to: ‘raw.tar.gz.2’

raw.tar.gz.2        100%[===================>]  97.46M  31.0MB/s    in 3.9s    

2020-12-19 11:40:23 (25.2 MB/s) - ‘raw.tar.gz.2’ saved [102198198/102198198]



In [9]:
!tar -xzf raw.tar.gz

In [10]:
my_frame = pd.read_csv('raw/raw', sep='\t')

In [11]:

my_frame.columns = ['en', 'jp']
my_frame = my_frame[:500000]
my_frame = my_frame[my_frame.columns[::-1]]

In [12]:
my_frame

,jp,en
0,俺の相手は シャークだ。,my opponent is shark.
1,引き換えだ ある事とある物の,this is one thing in exchange for another.
2,もういいよ ごちそうさま ううん,"yeah, i'm fine."
3,もう会社には来ないでくれ 電話もするな,don't come to the office anymore. don't call m...
4,きれいだ。,looks beautiful.
...,...,...
499995,FBIの男に強要されたのに,i was threatened by a guy from your office.
499996,シャーペン回すんやめてくれへん? 気が散んねん。,it's distracting.
499997,荒れた生態系に水を,"it provides a simple, inexpensive"
499998,全ての職員や運転手と話を,"i've talked to every morgue attendant, every m..."


In [13]:
segmenter.tokenize(my_frame['jp'][1])

['引き換え', 'だ', ' ', 'ある', '事', 'と', 'ある', '物', 'の']

In [14]:
[tok.text for tok in spacy_en.tokenizer(my_frame['en'][1])]

['this', 'is', 'one', 'thing', 'in', 'exchange', 'for', 'another', '.']

In [15]:
my_frame.to_csv('my_frame.csv', index=False)  

In [16]:
MAX_LEN = 25

def tokenize_jp(text):
    """
    Tokenizes JP text from a string into a list of strings
    """
    return segmenter.tokenize(text)[:MAX_LEN]


def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    res = [tok.text for tok in spacy_en.tokenizer(text)]
    return res[:MAX_LEN]

In [17]:
SRC = Field(tokenize=tokenize_en, init_token='<sos>', eos_token='<eos>')
TRG = Field(tokenize=tokenize_jp, init_token='<sos>', eos_token='<eos>')

In [18]:
dataset = TabularDataset(path='my_frame.csv', 
                         format='csv', 
                         fields=[ ('jp', TRG), ('en', SRC),],
                         skip_header=True)

In [19]:
train_data, valid_data, test_data = dataset.split(split_ratio=[0.7, 0.1, 0.2], 
                                            random_state=random.getstate())

In [20]:
SRC.build_vocab(train_data, min_freq=3)
TRG.build_vocab(train_data, min_freq=3)

In [21]:
print(vars(train_data.examples[0])['en'])

['dean', 'will', 'be', 'fine', '.']


In [22]:
print (len(SRC.vocab), len(TRG.vocab))
print (SRC.vocab.freqs.most_common(10))
print (TRG.vocab.freqs.most_common(10))

23548 34107
[('.', 230764), (',', 120757), ('the', 88969), ('you', 88482), ('i', 83759), ('?', 66455), ('to', 62365), ('a', 52027), ("'s", 49882), ('it', 49686)]
[(' ', 251987), ('の', 135247), ('は', 101010), ('に', 90906), ('て', 84292), ('を', 78047), ('が', 72143), ('た', 71440), ('?', 51927), ('。', 47713)]


In [23]:
# gpu = False
# device = torch.device('cuda' if gpu and torch.cuda.is_available() else 'cpu')

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 256

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
     batch_size = BATCH_SIZE,
     sort_key=lambda x: len(x.jp), 
     sort_within_batch=False,
     device = device)


In [25]:

# for b in valid_iterator:
#     print (b.jp, b.en)
#     sys.exit()


In [26]:
print(vars(valid_iterator))

{'batch_size': 256, 'train': False, 'dataset': <torchtext.data.dataset.Dataset object at 0x7f0f3dc27ac8>, 'batch_size_fn': None, 'iterations': 0, 'repeat': False, 'shuffle': False, 'sort': True, 'sort_within_batch': False, 'sort_key': <function <lambda> at 0x7f0f0f1ff488>, 'device': device(type='cpu'), 'random_shuffler': <torchtext.data.utils.RandomShuffler object at 0x7f0f0f5b63c8>, '_iterations_this_epoch': 0, '_random_state_this_epoch': None, '_restored_from_state': False}


In [27]:
print(len(valid_iterator))

391


In [28]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs  = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [29]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [30]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        # 128(batch_size)个例子, 每个例子里的句子长度为32(trg_len), 每个句子里的单词都有5893维(vacab_size)

        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [31]:
len(SRC.vocab), len(TRG.vocab)

(23548, 34107)

In [32]:
INPUT_DIM = len(SRC.vocab) # 7855
OUTPUT_DIM = len(TRG.vocab) # 5893
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.6
DEC_DROPOUT = 0.6

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [33]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(23548, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.6)
    (dropout): Dropout(p=0.6, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(34107, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.6)
    (fc_out): Linear(in_features=512, out_features=34107, bias=True)
    (dropout): Dropout(p=0.6, inplace=False)
  )
)

In [34]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 39,612,987 trainable parameters


In [35]:
optimizer = optim.Adam(model.parameters())

TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
pad_idx = TRG.vocab.stoi['<pad>']
print(TRG.pad_token)  # <pad>
print(TRG.vocab.stoi[TRG.pad_token]) # 1 

criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

<pad>
1


In [184]:
# epoch_loss = 0
# for i, batch in enumerate(train_iterator):
#   # print(i,batch)
#   src = batch.en
#   trg = batch.jp
    
#   output = model(src, trg)
#   output_dim = output.shape[-1]  # 5893

#   new_output = output[1:].view(-1, output_dim)  # remove <sos>, [out len * batchsize, output dim]
#   new_trg = trg[1:].view(-1)  # remove <sos> , [trg len * batchsize]
    
#   loss = criterion(new_output, new_trg)
#   epoch_loss += loss.item()
# print(epoch_loss / len(train_iterator)) # 3.0412618011104904

In [36]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.en
        trg = batch.jp
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [37]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.en
            trg = batch.jp

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [38]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [39]:
import time

In [ ]:
N_EPOCHS = 5
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

In [ ]:
def translate_sentence(sentence,src_field,trg_field,model,device, max_len=50):
    model.eval()

    if isinstance(sentence,str):
        nlp = spacy.load('en')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]

    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor)

    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    #
    for i in range(max_len):
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.decoder(trg_tensor, hidden, cell)
        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token)
        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

    return trg_tokens[1:-1] # remove <sos> and <eos>

In [ ]:
example_idx = 24

src = vars(train_data.examples[example_idx])['en']
trg = vars(train_data.examples[example_idx])['jp']

print(f'src = {src}')
print(f'trg = {trg}')

translation = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

model_save_name = 'tut1-model.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)